In [1]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa 05 do curso Como usar Python para descobrir uma rede de interesses
#

In [2]:
import pandas as pd
import requests
import json
import time

Fonte:
    
https://dadosabertos.camara.leg.br/swagger/api.html

Prefixo comum do site da API de proposicoes por ano

In [3]:
prefixo = 'https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano='

Exemplo de página gerada: 
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2010&ordem=ASC&ordenarPor=id&formato=json&pagina=94&itens=100

In [4]:
projetos = [] # cria lista vazia 

# Itera entre os anos de 2010 e 2018
for i in range(2010, 2019): 
    ano = str(i)
    
    # Forma a url precisa
    url = prefixo + ano + '&ordem=ASC&ordenarPor=id'
    print("ANO:", ano)
    print(url)
    
    # Seta os parâmetros para acessar a API
    parametros = {'formato': 'json', 'itens': 100}
    resposta = requests.get(url, parametros)
    
    # Itera em cada resposta a partir do itens links
    # Essa primeira vez apenas para coletar o número da última página
    for vez in resposta.json()['links']:
        conta = {"rel": vez['rel'].strip(), 
                   "href": vez['href'].strip()
                   }
        
    link_ultimo = str(conta['href'].strip())
    posicao = int(link_ultimo.find('&itens='))
    ultima = int(link_ultimo[107:posicao]) + 1
    print(ultima, " páginas")
    
    # Busca em cada página, agora com o número correto de páginas finais
    for pagina in range(1, ultima):
        # Passa os parâmetros 
        parametros = {'formato': 'json', 'itens': 100, 'pagina': pagina}
        resposta = requests.get(url, parametros)
        #print("página:", pagina)
        
        # Acessa cada resposta e preenche um dicionário com os dados
        for vez in resposta.json()['dados']:
            
            dicionario = {"id": str(vez['id']).strip(), 
                          "uri": str(vez['uri']).strip(), 
                          "siglaTipo": str(vez['siglaTipo']).strip(), 
                          "codTipo": str(vez['codTipo']).strip(), 
                          "numero": str(vez['numero']).strip(), 
                          "ano": str(vez['ano']).strip(), 
                          "ementa": str(vez['ementa']).strip()
                          }
            # Adiciona cada dicionário na lista 
            projetos.append(dicionario)

ANO: 2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2010&ordem=ASC&ordenarPor=id
95  páginas
ANO: 2011
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2011&ordem=ASC&ordenarPor=id
229  páginas
ANO: 2012
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2012&ordem=ASC&ordenarPor=id
153  páginas
ANO: 2013
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2013&ordem=ASC&ordenarPor=id
177  páginas
ANO: 2014
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2014&ordem=ASC&ordenarPor=id
103  páginas
ANO: 2015
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2015&ordem=ASC&ordenarPor=id
249  páginas
ANO: 2016
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2016&ordem=ASC&ordenarPor=id
160  páginas
ANO: 2017
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2017&ordem=ASC&ordenarPor=id
203  páginas
ANO: 2018
https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2018&ordem=ASC&ordenarPor=id
111  páginas


In [5]:
# A partir da lista acima cria um dataframe

In [6]:
df_projetos_api = pd.DataFrame(projetos)

In [7]:
df_projetos_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146505 entries, 0 to 146504
Data columns (total 7 columns):
ano          146505 non-null object
codTipo      146505 non-null object
ementa       146505 non-null object
id           146505 non-null object
numero       146505 non-null object
siglaTipo    146505 non-null object
uri          146505 non-null object
dtypes: object(7)
memory usage: 7.8+ MB


In [8]:
df_projetos_api.reset_index().tail()

,index,ano,codTipo,ementa,id,numero,siglaTipo,uri
146500,146500,2018,134,Submete à apreciação do Congresso Nacional o t...,2200230,715,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
146501,146501,2018,134,Submete à apreciação do Congresso Nacional o t...,2201562,789,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
146502,146502,2018,134,Submete à apreciação do Congresso Nacional o t...,2201777,773,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
146503,146503,2018,134,Submete à apreciação do Congresso Nacional o t...,2202329,787,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
146504,146504,2018,134,Submete à apreciação do Congresso Nacional o t...,2203698,760,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...


In [9]:
df_projetos_api.to_csv('resultados/projetos_camara_2010_2018.csv', index=False)

Opcional: para não ter que rodar todo o programa, se for executado aos poucos, 

Pode começar por aqui, se o programa tiver sido fechado

In [4]:
df_projetos_api = pd.read_csv("resultados/projetos_camara_2010_2018.csv",sep=',',\
                                             encoding = 'utf-8', \
                                             converters={'index': lambda x: str(x), 
                                                         'ano': lambda x: str(x), 
                                                         'ementa': lambda x: str(x),
                                                         'id': lambda x: str(x),
                                                         'codTipo': lambda x: str(x),
                                                         'numero': lambda x: str(x),
                                                         'siglaTipo': lambda x: str(x),
                                                         'uri': lambda x: str(x)
                                                        })

In [13]:
df_projetos_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146505 entries, 0 to 146504
Data columns (total 7 columns):
ano          146505 non-null object
codTipo      146505 non-null object
ementa       146505 non-null object
id           146505 non-null object
numero       146505 non-null object
siglaTipo    146505 non-null object
uri          146505 non-null object
dtypes: object(7)
memory usage: 7.8+ MB


In [14]:
df_projetos_api.reset_index().tail()

,index,ano,codTipo,ementa,id,numero,siglaTipo,uri
146500,146500,2018,134,Submete à apreciação do Congresso Nacional o t...,2200230,715,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
146501,146501,2018,134,Submete à apreciação do Congresso Nacional o t...,2201562,789,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
146502,146502,2018,134,Submete à apreciação do Congresso Nacional o t...,2201777,773,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
146503,146503,2018,134,Submete à apreciação do Congresso Nacional o t...,2202329,787,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
146504,146504,2018,134,Submete à apreciação do Congresso Nacional o t...,2203698,760,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...


Deixa o texto das ementas em maiúscula para fazer a busca

In [15]:
df_projetos_api['ementa'] = df_projetos_api['ementa'].str.upper()

Faz uma busca pelos termos que são ligados à industria farmaceutica

In [16]:
search_list = ['FARMÁCIA POPULAR',
               'ASSISTÊNCIA FARMACÊUTICA',
               'REMÉDIO',
               'FÁRMACOS',
               'MEDICAMENTOS',
               'FARMACÊUTICOS',
               'COMPLEXO INDUSTRIAL E INOVAÇÃO EM SAÚDE',
               'FARMACOQUÍMICO',
               'RENAME',
               'RELAÇÃO NACIONAL DE MEDICAMENTOS ESSENCIAIS',
               'FITOSSANITÁRIOS',
               'INSUMOS AGRÍCOLAS',
               'DEFENSIVOS AGRÍCOLAS',
               'AGROTÓXICOS',
               'PATENTE',
               'INPI',
               'INSTITUTO NACIONAL DA PROPRIEDADE INDUSTRIAL',
               'PDP',
               'PARCERIA PARA O DESENVOLVIMENTO PRODUTIVO',
               'SAÚDE INDÍGENA']

In [17]:
mask = df_projetos_api['ementa'].str.contains('|'.join(search_list))
farma_proj  = df_projetos_api[mask]

In [18]:
farma_proj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046 entries, 21 to 146244
Data columns (total 7 columns):
ano          1046 non-null object
codTipo      1046 non-null object
ementa       1046 non-null object
id           1046 non-null object
numero       1046 non-null object
siglaTipo    1046 non-null object
uri          1046 non-null object
dtypes: object(7)
memory usage: 65.4+ KB


In [19]:
farma_proj.reset_index().head()

,index,ano,codTipo,ementa,id,numero,siglaTipo,uri
0,21,2010,139,DISPÕE SOBRE A PROIBIÇÃO DE PROPAGANDA DE MEDI...,465521,6724,PL,https://dadosabertos.camara.leg.br/api/v2/prop...
1,165,2010,148,SOLICITA INFORMAÇÕES AO SENHOR MINISTRO DA SAÚ...,465806,4681,RIC,https://dadosabertos.camara.leg.br/api/v2/prop...
2,173,2010,139,"ALTERA O § 4º DO ART. 5º DA LEI Nº 6.360, DE 2...",465818,6745,PL,https://dadosabertos.camara.leg.br/api/v2/prop...
3,184,2010,139,INSTITUI A DISTRIBUIÇÃO GRATUITA DE MEDICAMENT...,465836,6756,PL,https://dadosabertos.camara.leg.br/api/v2/prop...
4,539,2010,134,SUBMETE À DELIBERAÇÃO DO CONGRESSO NACIONAL O ...,466405,1127,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...


Acessa um arquivo da Câmara com os tipos de proposicoes, para orientacao

In [20]:
with open('resultados/response_1547490977470.json',  encoding='utf-8') as json_data:
    d = json.load(json_data)

lista = d['dados']

df_tipos = pd.DataFrame(lista, columns = ['id', 'sigla', 'nome', 'descricao'])

In [21]:
df_tipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 4 columns):
id           420 non-null object
sigla        420 non-null object
nome         420 non-null object
descricao    411 non-null object
dtypes: object(4)
memory usage: 13.2+ KB


In [22]:
df_tipos.reset_index().head()

,index,id,sigla,nome,descricao
0,0,129,CON,Consulta,Consulta
1,1,130,EMC,Emenda na Comissão,Emenda Apresentada na Comissão
2,2,131,EMP,Emenda de Plenário,Emenda de Plenário
3,3,132,EMS,Emenda/Substitutivo do Senado,Emenda/Substitutivo do Senado
4,4,133,INC,Indicação,Indicação


In [23]:
df_tipos.to_excel('resultados/tipos_proposicoes_camara.xlsx',sheet_name='Sheet1')

Com o dataframe selecionado agora acessa outro ponto da API

Agora para coletar os detalhes de cada proposicao

Neste caso 1.046 vezes

In [26]:
prefixo = 'https://dadosabertos.camara.leg.br/api/v2/proposicoes/'

In [27]:
projetos = []

for num, row in farma_proj.iterrows():
    chave = (row['id']).strip()
    url = prefixo + chave 
    print(url)
    parametros = {'formato': 'json'}
    resposta = requests.get(url, parametros)
    
    # Faz uma pequena pausa de precaucao
    # Para servidor não barrar
    time.sleep(1)
    
    vez = resposta.json()['dados']
    
    dicionario = {"id": vez['id'], 
                      "uri": vez['uri'], 
                      "siglaTipo": vez['siglaTipo'], 
                      "codTipo": vez['codTipo'], 
                      "numero": vez['numero'], 
                      "ano": vez['ano'], 
                      "ementa": vez['ementa'], 
                      "dataApresentacao": vez['dataApresentacao'], 
                      "uriOrgaoNumerador": vez['uriOrgaoNumerador'], 
                      "uriUltimoRelator": vez["uriUltimoRelator"] if "uriUltimoRelator" in vez else None, 
                      "status_datahora": vez['statusProposicao']['dataHora'], 
                      "status_sequencia": vez['statusProposicao']['sequencia'], 
                      "status_siglaOrgao": vez['statusProposicao']['siglaOrgao'], 
                      "status_uriOrgao": vez['statusProposicao']['uriOrgao'], 
                      "status_regime": vez['statusProposicao']['regime'], 
                      "status_descricaoTramitacao": vez['statusProposicao']['descricaoTramitacao'], 
                      "status_idTipoTramitacao": vez['statusProposicao']['codTipoTramitacao'], 
                      "status_descricaoSituacao": vez['statusProposicao']['descricaoSituacao'], 
                      "status_CodSituacao": vez['statusProposicao']['codSituacao'], 
                      "status_despacho": vez['statusProposicao']['despacho'], 
                      "status_url": vez['statusProposicao']['url'], 
                      "uriAutores": vez['uriAutores'], 
                      "descricaoTipo": vez['descricaoTipo'], 
                      "ementaDetalhada": vez['ementaDetalhada'], 
                      "keywords": vez['keywords'], 
                      "uriPropPrincipal": vez['uriPropPrincipal'], 
                      "uriPropAnterior": vez['uriPropAnterior'], 
                      "uriPropPosterior": vez['uriPropPosterior'], 
                      "urlInteiroTeor": vez['urlInteiroTeor'], 
                      "urnFinal": vez['urnFinal'], 
                      "texto": vez['texto'], 
                      "justificativa": vez['justificativa']
                      }
    projetos.append(dicionario)

https://dadosabertos.camara.leg.br/api/v2/proposicoes/465521
https://dadosabertos.camara.leg.br/api/v2/proposicoes/465806
https://dadosabertos.camara.leg.br/api/v2/proposicoes/465818
https://dadosabertos.camara.leg.br/api/v2/proposicoes/465836
https://dadosabertos.camara.leg.br/api/v2/proposicoes/466405
https://dadosabertos.camara.leg.br/api/v2/proposicoes/467292
https://dadosabertos.camara.leg.br/api/v2/proposicoes/467836
https://dadosabertos.camara.leg.br/api/v2/proposicoes/467887
https://dadosabertos.camara.leg.br/api/v2/proposicoes/467928
https://dadosabertos.camara.leg.br/api/v2/proposicoes/468424
https://dadosabertos.camara.leg.br/api/v2/proposicoes/469290
https://dadosabertos.camara.leg.br/api/v2/proposicoes/471646
https://dadosabertos.camara.leg.br/api/v2/proposicoes/471973
https://dadosabertos.camara.leg.br/api/v2/proposicoes/472083
https://dadosabertos.camara.leg.br/api/v2/proposicoes/472122
https://dadosabertos.camara.leg.br/api/v2/proposicoes/473493
https://dadosabertos.cam

Cria novo dataframe com os dados coletados

In [35]:
df_projetos_farma_api = pd.DataFrame(projetos)

In [36]:
df_projetos_farma_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 32 columns):
ano                           1046 non-null int64
codTipo                       1046 non-null int64
dataApresentacao              1046 non-null object
descricaoTipo                 1046 non-null object
ementa                        1046 non-null object
ementaDetalhada               648 non-null object
id                            1046 non-null int64
justificativa                 0 non-null object
keywords                      648 non-null object
numero                        1046 non-null int64
siglaTipo                     1046 non-null object
status_CodSituacao            932 non-null float64
status_datahora               1046 non-null object
status_descricaoSituacao      932 non-null object
status_descricaoTramitacao    1046 non-null object
status_despacho               1046 non-null object
status_idTipoTramitacao       1046 non-null object
status_regime                 1046 n

In [30]:
df_projetos_farma_api.to_csv('resultados/projetos_farma_camara_2010_2018.csv', index=False)

In [31]:
df_projetos_farma_api.to_excel('resultados/projetos_farma_camara_2010_2018.xlsx',sheet_name='Sheet1')

In [32]:
# Programa também pode começar por aqui, caso a intenção seja rodar aos poucos

In [33]:
df_projetos_farma_api = pd.read_excel("resultados/projetos_farma_camara_2010_2018.xlsx",\
                                      sheet_name='Sheet1',
                                      converters={'ano': lambda x: str(x), 
                                                  'id': lambda x: str(x), 
                                                  'idTipo': lambda x: str(x),
                                                  'numero': lambda x: str(x),
                                                  'status_idSituacao': lambda x: str(x),
                                                  'status_idTipoTramitacao': lambda x: str(x),
                                                  'status_sequencia': lambda x: str(x)
                                                  })

In [37]:
df_projetos_farma_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 32 columns):
ano                           1046 non-null int64
codTipo                       1046 non-null int64
dataApresentacao              1046 non-null object
descricaoTipo                 1046 non-null object
ementa                        1046 non-null object
ementaDetalhada               648 non-null object
id                            1046 non-null int64
justificativa                 0 non-null object
keywords                      648 non-null object
numero                        1046 non-null int64
siglaTipo                     1046 non-null object
status_CodSituacao            932 non-null float64
status_datahora               1046 non-null object
status_descricaoSituacao      932 non-null object
status_descricaoTramitacao    1046 non-null object
status_despacho               1046 non-null object
status_idTipoTramitacao       1046 non-null object
status_regime                 1046 n

Agora cria um dataframe com os autores - nomes primários

In [38]:
autores = []

prefixo = 'https://dadosabertos.camara.leg.br/api/v2/proposicoes/'

for num, row in df_projetos_farma_api.iterrows():
    chave = (str(row['id'])).strip()
    ano = (str(row['ano'])).strip()
    numero = (str(row['numero'])).strip()
    projeto = (row['siglaTipo']).strip() + ' ' + numero + '/' + ano
    print(projeto)
    url = prefixo + chave + '/autores'
    print(url)
    time.sleep(1)
    
    parametros = {'formato': 'json'}
    resposta = requests.get(url, parametros)
    #print(resposta)
    for vez in resposta.json()['dados']:
        dicionario = {"parlamentar": vez['nome'], 
                          "uri": vez['uri'], 
                          "projeto": projeto, 
                          "id_proposicao": str(chave), 
                          "tipo": vez['tipo'],
                          "cod_tipo": str(vez['codTipo'])
                          }
        autores.append(dicionario)

PL 6724/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/465521/autores
RIC 4681/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/465806/autores
PL 6745/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/465818/autores
PL 6756/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/465836/autores
MSC 1127/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/466405/autores
PL 6859/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/467292/autores
REQ 6360/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/467836/autores
PL 6900/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/467887/autores
PL 6906/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/467928/autores
INC 6212/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/468424/autores
REQ 519/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/469290/autores
REQ 21/2010
https://dadosabertos.camara.leg.br/api/v2/proposicoes/471646/autores
REQ 658/2010


In [39]:
df_projetos_farma_autores = pd.DataFrame(autores)

In [40]:
df_projetos_farma_autores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 6 columns):
cod_tipo         1461 non-null object
id_proposicao    1461 non-null object
parlamentar      1461 non-null object
projeto          1461 non-null object
tipo             1461 non-null object
uri              1357 non-null object
dtypes: object(6)
memory usage: 68.6+ KB


In [41]:
df_projetos_farma_autores.reset_index().head()

,index,cod_tipo,id_proposicao,parlamentar,projeto,tipo,uri
0,0,10000,465521,Luiz Couto,PL 6724/2010,Deputado,https://dadosabertos.camara.leg.br/api/v2/depu...
1,1,10000,465806,Beto Faro,RIC 4681/2010,Deputado,https://dadosabertos.camara.leg.br/api/v2/depu...
2,2,40000,465818,Senado Federal - Osmar Dias,PL 6745/2010,Órgão do Poder Legislativo,None
3,3,40000,465836,Senado Federal - Expedito Júnior,PL 6756/2010,Órgão do Poder Legislativo,None
4,4,30000,466405,Poder Executivo,MSC 1127/2010,Órgão do Poder Executivo,None


In [42]:
df_projetos_farma_autores.to_excel('resultados/autores_projetos_farma_camara_2010_2018.xlsx',sheet_name='Sheet1')

Mas normalmente em muitos casos os projetos que têm como autor deputados têm mais de um autor

Por isso é necessário vasculhar de novo para preencher todos

In [43]:
autores_completo = []


for num, row in df_projetos_farma_autores.iterrows():
    tipo = (row['tipo']).strip()
    cod_tipo = str(row['cod_tipo'].strip())
    id_proposicao = str(row['id_proposicao'].strip())
    parlamentar = str(row['parlamentar'].strip())
    uri_deputado = row['uri']
    projeto = str(row['projeto'].strip())
    print(projeto)
    #print(tipo)

    
    if tipo == "Deputado":
        url = (row['uri']).strip()
        print(url)
    
        parametros = {'formato': 'json'}
        resposta = requests.get(url, parametros)
        time.sleep(1)
        dicionario = {"cod_tipo": str(cod_tipo), 
                          "id_proposicao": str(id_proposicao), 
                          "parlamentar": parlamentar, 
                          "projeto": projeto, 
                          "tipo": tipo, 
                          "uri_deputado": uri_deputado,
                          "id_deputado": str(resposta.json()['dados']['id']),
                          "nome_completo": resposta.json()['dados']['nomeCivil'],
                          "idLegislatura": str(resposta.json()['dados']['ultimoStatus']['idLegislatura']),
                          "urlFoto": resposta.json()['dados']['ultimoStatus']['urlFoto'],
                          "data_nascimento": resposta.json()['dados']['dataNascimento']
                          }
        autores_completo.append(dicionario)
    # Caso o autor não seja deputado apenas preenche com dados já coletados 
    else:
        dicionario = {"cod_tipo": str(cod_tipo), 
                          "id_proposicao": str(id_proposicao), 
                          "parlamentar": parlamentar, 
                          "projeto": projeto, 
                          "tipo": tipo, 
                          "uri_deputado": uri_deputado,
                          "id_deputado": "nulo",
                          "nome_completo": "nulo",
                          "idLegislatura": "nulo",
                          "urlFoto": "nulo",
                          "data_nascimento": "nulo"
                          }
        autores_completo.append(dicionario)

PL 6724/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/74041
RIC 4681/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/141335
PL 6745/2010
PL 6756/2010
MSC 1127/2010
PL 6859/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/141428
REQ 6360/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/141556
PL 6900/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/141373
PL 6906/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/141548
INC 6212/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/133373
REQ 519/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/74238
REQ 21/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/74041
REQ 21/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/141438
REQ 658/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/73938
REQ 537/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/73937
REQ 537/2010
https://dadosabertos.camara.leg.br/api/v2/deputados/141335
REQ 319/2010
https://dadosab

In [44]:
df_projetos_farma_autores_completo = pd.DataFrame(autores_completo)

In [45]:
df_projetos_farma_autores_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 11 columns):
cod_tipo           1461 non-null object
data_nascimento    1461 non-null object
idLegislatura      1461 non-null object
id_deputado        1461 non-null object
id_proposicao      1461 non-null object
nome_completo      1461 non-null object
parlamentar        1461 non-null object
projeto            1461 non-null object
tipo               1461 non-null object
uri_deputado       1357 non-null object
urlFoto            1461 non-null object
dtypes: object(11)
memory usage: 125.6+ KB


In [37]:
df_projetos_farma_autores_completo.to_excel('resultados/autores_nomescompletos_projetos_farma_camara_2010_2018.xlsx',sheet_name='Sheet1')

In [42]:
df_projetos_farma_autores_completo.reset_index().head()

,index,cod_tipo,data_nascimento,idLegislatura,id_deputado,id_proposicao,nome_completo,parlamentar,projeto,tipo,uri_deputado,urlFoto
0,0,10000,1945-02-13,55,74041,465521,LUIZ ALBUQUERQUE COUTO,Luiz Couto,PL 6724/2010,Deputado,https://dadosabertos.camara.leg.br/api/v2/depu...,https://www.camara.leg.br/internet/deputado/ba...
1,1,10000,1969-06-20,56,141335,465806,JOSÉ ROBERTO OLIVEIRA FARO,Beto Faro,RIC 4681/2010,Deputado,https://dadosabertos.camara.leg.br/api/v2/depu...,https://www.camara.leg.br/internet/deputado/ba...
2,2,40000,nulo,nulo,nulo,465818,nulo,Senado Federal - Osmar Dias,PL 6745/2010,Órgão do Poder Legislativo,None,nulo
3,3,40000,nulo,nulo,nulo,465836,nulo,Senado Federal - Expedito Júnior,PL 6756/2010,Órgão do Poder Legislativo,None,nulo
4,4,30000,nulo,nulo,nulo,466405,nulo,Poder Executivo,MSC 1127/2010,Órgão do Poder Executivo,None,nulo


Cria campo codigo_unico para facilitar contagem

In [46]:
df_projetos_farma_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 32 columns):
ano                           1046 non-null int64
codTipo                       1046 non-null int64
dataApresentacao              1046 non-null object
descricaoTipo                 1046 non-null object
ementa                        1046 non-null object
ementaDetalhada               648 non-null object
id                            1046 non-null int64
justificativa                 0 non-null object
keywords                      648 non-null object
numero                        1046 non-null int64
siglaTipo                     1046 non-null object
status_CodSituacao            932 non-null float64
status_datahora               1046 non-null object
status_descricaoSituacao      932 non-null object
status_descricaoTramitacao    1046 non-null object
status_despacho               1046 non-null object
status_idTipoTramitacao       1046 non-null object
status_regime                 1046 n

In [47]:
def limpa1(linha):    
    return (linha['siglaTipo']).strip() + ' ' + str(linha['numero']).strip() + '/' + str(linha['ano']).strip()

df_projetos_farma_api["proposicao"] = df_projetos_farma_api.apply(limpa1, axis=1)

In [48]:
def limpa2(linha):    
    return (linha['proposicao']).strip() + ' _ ' + str(linha['id']).strip()

df_projetos_farma_api["codigo_unico"] = df_projetos_farma_api.apply(limpa2, axis=1)

In [49]:
#df_projetos_farma_api[['proposicao', 'codigo_unico']]

In [50]:
def limpa3(linha):    
    return (linha['projeto']).strip() + ' _ ' + str((linha['id_proposicao']).strip()) 

df_projetos_farma_autores_completo["codigo_unico"] = df_projetos_farma_autores_completo.apply(limpa3, axis=1)

In [51]:
#df_projetos_farma_autores_completo[['id_proposicao', 'codigo_unico']]

In [52]:
df_projetos_farma_autores_completo.reset_index().head()

,index,cod_tipo,data_nascimento,idLegislatura,id_deputado,id_proposicao,nome_completo,parlamentar,projeto,tipo,uri_deputado,urlFoto,codigo_unico
0,0,10000,1945-02-13,55,74041,465521,LUIZ ALBUQUERQUE COUTO,Luiz Couto,PL 6724/2010,Deputado,https://dadosabertos.camara.leg.br/api/v2/depu...,https://www.camara.leg.br/internet/deputado/ba...,PL 6724/2010 _ 465521
1,1,10000,1969-06-20,56,141335,465806,JOSÉ ROBERTO OLIVEIRA FARO,Beto Faro,RIC 4681/2010,Deputado,https://dadosabertos.camara.leg.br/api/v2/depu...,https://www.camara.leg.br/internet/deputado/ba...,RIC 4681/2010 _ 465806
2,2,40000,nulo,nulo,nulo,465818,nulo,Senado Federal - Osmar Dias,PL 6745/2010,Órgão do Poder Legislativo,None,nulo,PL 6745/2010 _ 465818
3,3,40000,nulo,nulo,nulo,465836,nulo,Senado Federal - Expedito Júnior,PL 6756/2010,Órgão do Poder Legislativo,None,nulo,PL 6756/2010 _ 465836
4,4,30000,nulo,nulo,nulo,466405,nulo,Poder Executivo,MSC 1127/2010,Órgão do Poder Executivo,None,nulo,MSC 1127/2010 _ 466405


In [53]:
df_projetos_farma_autores_completo['parlamentar'].value_counts().head(20)

Marcelo Belinati                                                                                                                               67
Padre João                                                                                                                                     24
Comissão de Seguridade Social e Família                                                                                                        18
João Daniel                                                                                                                                    16
Nilto Tatto                                                                                                                                    16
Jandira Feghali                                                                                                                                15
Luiz Nishimori                                                                                                              

In [54]:
df_projetos_farma_autores_completo['nome_completo'].value_counts().head(20)

nulo                                        104
MARCELO BELINATI MARTINS                     67
JOÃO CARLOS SIQUEIRA                         26
NILTO IGNACIO TATTO                          16
JOAO SOMARIVA DANIEL                         16
JANDIRA FEGHALI                              15
CARMEN EMÍLIA BONFÁ ZANOTTO                  13
LUIZ HILOSHI NISHIMORI                       13
ALEXANDRE LEITE DA SILVA                     12
NILSON APARECIDO LEITÃO                      12
FLÁVIA CARREIRO ALBUQUERQUE MORAIS           12
LEANDRE DAL PONTE                            12
FLORISVALDO FIER                             11
ALESSANDRO LUCCIOLA MOLON                    11
MARCUS VINÍCIUS CAETANO PESTANA DA SILVA     11
LUIS CARLOS HEINZE                           10
JOSÉ SARNEY FILHO                            10
LUIZ PAULO TEIXEIRA FERREIRA                  9
JORGE JOSE SANTOS PEREIRA SOLLA               9
AMAURI SANTOS TEIXEIRA                        9
Name: nome_completo, dtype: int64

In [55]:
df_projetos_farma_api.to_excel('resultados/projetos_farma_camara_2010_2018.xlsx',sheet_name='Sheet1')
df_projetos_farma_autores_completo.to_excel('resultados/autores_nomescompletos_projetos_farma_camara_2010_2018.xlsx',sheet_name='Sheet1')